In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/crowdflower-weather-twitter/train.csv")
test = pd.read_csv("/kaggle/input/crowdflower-weather-twitter/test.csv")
all_data = pd.concat([train,test])

In [ ]:
pd.options.display.max_columns = 1000
all_data

In [ ]:
variable = pd.read_table("/kaggle/input/crowdflower-weather-twitter/variableNames.txt",header=None)
variable


In [ ]:
variable

In [ ]:
var = "/kaggle/input/crowdflower-weather-twitter/variableNames.txt"

In [ ]:
with open(var) as f:    #open - 파일 입출력?
    print(f.read())
# https://dojang.io/mod/page/view.php?id=2325

In [ ]:
all_data.iloc[0]

In [ ]:
all_data.loc[0]

In [ ]:
for i in range(20):
    print(i, ' : ', all_data['tweet'].iloc[i])

In [ ]:
all_data = all_data.fillna("nan")


In [ ]:
all_data["tweet"] = all_data["tweet"] + " " + all_data["state"] + " " + all_data["location"]


In [ ]:
all_data["tweet"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer   #count를 기반으로..
tfidf = TfidfVectorizer(min_df=4,ngram_range=(1, 2),sublinear_tf=True)
text = tfidf.fit_transform(all_data["tweet"])
# min_df=1(최소단어등장횟수) 최소3은 넣어주자
# ngram_range=(1, 1), 윈도우사이즈 (conv1느낌) 한번에 단어를 인식을할때 몇개씩 인식할거냐
# sublinear_tf=False 칼럼마다 스케일링해주는것

In [ ]:
#트리모델 안쓰고 선형모델,딥러닝 쓸거기때문에 차원축소(svd.. 이런거) 따로 안해도됨.

In [ ]:
text #120103:데이터개수 / 74398: 단어의개수(칼럼의개수) / 3234262: 0101애들중에 1인 애들의 개수


In [ ]:
train_text = text[:len(train)]
test_text = text[len(train):]

In [ ]:
from sklearn.linear_model import Ridge  
#예측을 한 컬럼만 해주는게 아니라 여러개의 칼럼을 동시 예측해주는데, regression을 여러번 실행하는데 도움을 주는
from sklearn.multioutput import RegressorChain
rc = RegressorChain(Ridge(alpha=5, random_state=1234),random_state=1234) 
#alpha(규제항-과적합 예방) 디폴트는 1
rc.fit(train_text,train.iloc[:,4:])  #train에서 

result = rc.predict(test_text)
result

In [ ]:
#result값을 보면 <0, 1< 인 값들이 존재하는데, 우리는 0~1사이의 값을 예측해야하기 때문에 (0,1)값으로 변환해준다.
#softmax를 적용해봐도 되지 않았을까...?(0~1사이값으로 만들어주니까..?)
result = np.where(result < 0, 0, result)  
result = np.where(result > 1, 1, result)
result

In [ ]:
sub = pd.read_csv("/kaggle/input/crowdflower-weather-twitter/sampleSubmission.csv")
sub.iloc[:,1:] = result

############# 여기부턴 딥러닝 ###############

In [ ]:
# 토크나이저 선언
from keras.preprocessing.text import Tokenizer
tk = Tokenizer()

In [ ]:
# 정수인코딩
# 코퍼스의 모든 단어를 숫자로 매핑
tk.fit_on_texts(all_data["tweet"])
tk.word_index # 빈도순 # 빈도순

In [ ]:
tk.word_counts

In [ ]:
len(tk.word_index) # 단어의 총 개수(종류 개수)

In [ ]:
all_text = tk.texts_to_sequences(all_data["tweet"]) # 각 텍스트를 숫자로
all_text

In [ ]:
from keras.preprocessing.sequence import pad_sequences
all_pad = pad_sequences(all_text,)  #문장 길이 모두 맞춰주기 위해 '0'으로 채워준다.
all_pad

In [ ]:
all_pad.shape #가장 길었던 애가 단어 48개

In [ ]:
train_pad = all_pad[:len(train)]
test_pad = all_pad[len(train):]

In [ ]:
veec = pd.read_table("../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec")

In [ ]:
veec

In [ ]:
#pre-trained된 embedding을 사용해볼 것이다.
#가져온 벡터를 dict형태로 바꿔주는 함수
def load_embedding(path):
    embeddings = {}
    with open(path) as f:
        for i in f:
            value = i.rstrip().split() #rstrip 맨 오른쪽 띄어쓰기 지워줌
            word = value[0]
#             vector = value[1:] #이게 램 많이 잡아먹는코드 #그냥하면 2백만개가져와서 ram터짐
            vector = np.asarray(value[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

In [ ]:
embeddings1 = load_embedding("../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec")

In [ ]:
tk.word_index

In [ ]:
# 가져온 pre-trained 임베딩 전체 단어 중 여기 코퍼스에 있는 단어만 뽑아와주는 함수.
def filter_embedding(embeddings, word_index, vocab_size, dim):
    embedding_matrix = np.zeros([vocab_size,dim])  #초기화 -초기값으로 설정(초기값='0')
    for word, i in word_index.items():
        vector = embeddings.get(word)  
        # embeddings : 우리가 가져온 pretrained embedding
        # embedding에 사용된 대용량의 word들을 가져와 저장(리스트 형태로)
        if vector is not None:
            embedding_matrix[i] = vector
    return embedding_matrix

In [ ]:
embedding_matrix = filter_embedding(embeddings1, tk.word_index, len(tk.word_index)+1,300)

In [ ]:
embedding_matrix

In [ ]:
from keras import Sequential
from keras.layers import *
model = Sequential()
# model.add(Embedding(len(tk.word_index)+1,10,input_length=all_pad.shape[1]))
model.add(Embedding(len(tk.word_index)+1,300,input_length=all_pad.shape[1],weights=[embedding_matrix],trainable=False))
# weights 에 pretrained embedding으로 얻은 embedding_matrix로 사용
# trainable = False : pre-trained 모델 학습은 실행하지 않도록 fix해놓는것
model.add(GRU(128)) #LSTM의 단점을 보완하고 속도를 향상시킨 방법론 GRU
model.add(Dense(64,activation="relu"))

#출력층
model.add(Dense(24)) #예측해야할것 column 개수가 24개, #activation의 디폴트는 linear임
model.compile(optimizer="adam",
              loss="mse") # 이번엔 classification이 아니라 regression이기 때문에 loss function은 mse로
model.fit(train_pad,train.iloc[:,4:],batch_size=64,epochs=5)

In [ ]:
result_text = model.predict(test_pad)
result_text = np.where(result_text < 0, 0, result_text)
result_text = np.where(result_text > 1, 1, result_text)
# result_text

In [ ]:
sub = pd.read_csv("/kaggle/input/crowdflower-weather-twitter/sampleSubmission.csv")
sub.iloc[:,1:] = result_text
# 앙상블
sub.iloc[:,1:] = 0.8*result_text + 0.2*result
sub.to_csv("submission_ensemble.csv",index = 0)

# pre-trained embedding관련 참고하면 매우 유용
# https://dacon.io/competitions/official/235670/codeshare/1892?dtype=recent